In [1]:
import pandas as pd 
df = pd.read_csv('data.csv')
df.tail()

,Year,Film,Song,Composer,Language
145,2019,Manoharam,Akale,Sanjeev T,Malyalam
146,2020,Gauthamante Radham,Uyire,Ankit Menon,Malyalam
147,2020,Maniyarayile Ashokan,Olu,Sreehari K Nair,Malyalam
148,2021,Thala,Poonkodiye,Ankit Menon,Malyalam
149,2021,Kudukku 2025,Maaran,Bhoomee,Malyalam


In [72]:
# How many songs has he outputted ? 
# df['Year'].value_counts().sort_values().plot(kind = 'barh')

In [73]:
# # How many songs has he outputted ? 
# df['Composer'].value_counts().sort_values(ascending=False)[0:10].plot(kind = 'barh')


In [4]:
# with open('test_kadal_adiye.txt', 'r', encoding='utf8') as file:
#     data = file.read()
    
# data.split('\n')

In [5]:
queries = ['https://www.google.com/search?q= sid sriram ' + item + ' lyrics' for item in df['Song']]

with open('queries.txt', 'w') as f:
    for item in queries:
        f.write("%s\n" % item)

In [6]:
########################################################
########################################################
########################################################
########################################################
# add lyrics files 
########################################################
########################################################
########################################################
########################################################

In [7]:
# compare
# see how many filenames match 
# 146 of 150 not bad 

import glob
files = (glob.glob("lyrics/*"))
compare = [item.replace("sid sriram ","").replace("lyrics\\","").replace("lyrics\\sid sriram ","").replace(" lyrics.txt","") for item in files]
print(len(list(set(list(df['Song'])).intersection(compare))))
print(len(df))

146
150


In [8]:
# figure out matches for text files in db 
songs = list(df['Song'])

# files = (glob.glob("lyrics/*"))
dict_temp = {}
for i in range(0,len(songs)):
    for j in range(0,len(files)):
        parsed = files[j].replace("sid sriram ","").replace("lyrics\\","").replace("lyrics\\sid sriram ","").replace(" lyrics.txt","")
        if parsed == songs[i]:
            dict_temp[songs[i]] = files[j] 
#             print(files[j] + '_____' + songs[i])
matchez = pd.Series(dict_temp).to_frame('location').reset_index(drop=False).rename(columns={'index': 'Song'})

In [9]:
# merge on Song name 
merged = pd.merge(left=df, right=matchez, left_on='Song', right_on='Song')


In [11]:
all_files = []

for item in merged['location']:    
    with open(item, encoding='utf-8') as f:
        lines = f.readlines()
    all_files.append(lines)
    
# remove Male : and Female : 
# remove empty lines 

for i in range(0,len(all_files)):
    for j in range(0,len(all_files[i])):
        all_files[i][j] =all_files[i][j].strip().replace('…','.').replace('..','.').replace('  ',' ').replace('  ',' ').replace('Female :','').replace('\n','').replace('Male :','')

merged['content'] = all_files

# (x5) (x3) etc have to be made into their own lines

In [12]:
merged['original_line_count'] = [len(item) for item in merged['content'] ]
merged['unique_line_count'] = [len(list(set(item))) for item in merged['content'] ]
merged = merged.reset_index(drop=True)

In [13]:
# percentage of unique to regular lines 
# something to do with words in a song 

ind_word = []
for i in range(0, len(merged['content'])):
    temp = []
    for j in range(0,len(merged['content'][i])):
#         print(merged['content'][i][j].split(' '))
        temp.extend(merged['content'][i][j].split(' '))
#     print('______________')
    ind_word.append(temp)

    
merged['ind_word'] = ind_word

In [14]:
merged['original_word_count'] = [len(item) for item in merged['ind_word'] ]
merged['unique_word_count'] = [len(list(set(item))) for item in merged['ind_word'] ]

# percentage of lines copied 
# percentage of words copied 

merged['percent_of_lines_copied'] = (((merged['original_line_count'] - merged['unique_line_count']) / merged['original_line_count']) * 100)
merged['percent_of_words_copied'] = ((merged['original_word_count'] - merged['unique_word_count']) / merged['original_word_count']) * 100 

In [82]:
# merged.to_csv('merged.csv')

# The number of lines per song on average 
# The number of words per song on average 
# average number of unique lines 
# average number of unique words

# print(merged.original_line_count.describe()['mean'])
# print(merged.unique_line_count.describe()['mean'])
# print(merged.original_word_count.describe()['mean'])
# print(merged.unique_word_count.describe()['mean'])


# lexical richness of each artist (i.e., what proportion of the words used in the songs are distinct)


merged['lexical_richness'] = ((merged['original_word_count'] - merged['unique_word_count']) / merged['original_word_count'] ) * 100 

merged.lexical_richness.describe()


count    146.000000
mean      42.653953
std       13.831092
min        3.508772
25%       34.984956
50%       43.527967
75%       53.314195
max       74.590164
Name: lexical_richness, dtype: float64

In [77]:

# In what language does he copy the most 
# get weighted percentage based on numhber of songs in each language 
# merged.groupby(['Language']).mean()[['percent_of_lines_copied','percent_of_words_copied']]


# In what year does he copy the most 
# has the copying gotten worse or better over time 
# merged.groupby(['Year']).mean()[['percent_of_lines_copied','percent_of_words_copied']]

# merged.groupby(['Year','Language']).mean()[['percent_of_lines_copied','percent_of_words_copied']]


,percent_of_lines_copied,percent_of_words_copied
Year,,
2013,59.352774,58.971440
2015,40.842803,53.449937
2016,27.564348,39.491657
2017,28.950294,45.449604
2018,37.936889,44.397566
2019,28.637164,36.807407
2020,38.956607,47.915118
2021,37.302811,41.474677


In [117]:
# NOT WORKING :/ 
# !pip install googletrans
# import googletrans
# from googletrans import Translator
# translator = Translator()
# result = translator.translate('Mikä on nimesi', src='fi')

'పచ్చ పచ్చ పచ్చి మట్టి బొమ్మలా'

In [119]:

# # Does not do tamil or telugu - o well :/ 
# !pip install translate
# from translate import Translator
# translator= Translator(to_lang="English")
# translation = translator.translate(str(merged.content[82][1]))
# print(translation)


In [120]:
# can build some kindof lstm model , but before that need to get everything into english

# Use selenium to get into google translate and copy over the content 




'పచ్చ పచ్చ పచ్చి మట్టి బొమ్మలా'

In [66]:

# # agverage sid sriram song 
# from collections import Counter

    
# all_tel = []
# for i in range(0,len(merged)):
#     if merged['Language'][i]=='Telugu':
#         all_tel.extend(merged['content'][i])

# (Counter(all_tel).most_common())[0:15]

In [71]:

# # most common words used - dump as addition to most common lines 
# from collections import Counter

    
# all_tel = []
# for i in range(0,len(merged)):
#     if merged['Language'][i]=='Telugu':
#         all_tel.extend(merged['ind_word'][i])

# (Counter(all_tel).most_common())[0:100]

In [139]:
# eh , pass . too much effort 
# # ! pip install webdriver_manager 
# # ! pip install pyautogui 

# import requests 
# import json 
# import base64 
# from selenium import webdriver 
# from webdriver_manager.chrome import ChromeDriverManager 
# from selenium.webdriver.chrome.options import Options 
# from webdriver_manager.utils import ChromeType 
# import os 
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC 
# from selenium.webdriver.common.by import By 
# from bs4 import BeautifulSoup 
# from selenium.webdriver.support.ui import Select 
# import time 
# def driver_obj():
#     chrome_options_list = ['--no-sandbox','--disable-dev-shm-usage','--disable-gpu', '--start-maximized','--disable-infobars','--disable-extensions']
#     chrome_options = Options()
#     for i in range(0,len(chrome_options_list)):
#         chrome_options.add_argument(chrome_options_list[i])
#     driver = webdriver.Chrome(ChromeDriverManager(latest_release_url='https://chromedriver.storage.googleapis.com/LATEST_RELEASE',url='https://chromedriver.storage.googleapis.com').install(), options=chrome_options)
#     driver.maximize_window()
#     return driver


# driver = driver_obj()
# driver.get('https://www.google.com/search?q=' + 'google translate')
# time.sleep(2)
# aa = BeautifulSoup(driver.page_source, "html.parser")

# import time 
# import pyautogui
# pyautogui.click(365,457) 
# time.sleep(5)


# import pyperclip
# pyperclip.copy("పచ్చ పచ్చ పచ్చి మట్టి బొమ్మలా")

# pyautogui.keyDown('ctrl')
# pyautogui.press('v')
# pyautogui.keyUp('ctrl')  


# time.sleep(3)
# pyautogui.click(549,559) 

# time.sleep(3)
# spam = pyperclip.paste()
# print(spam)

# time.sleep(10)
# driver.quit()




[WDM] - Current google-chrome version is 92.0.4515
[WDM] - Get LATEST driver version for 92.0.4515
[WDM] - Driver [C:\Users\raosa\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


 
పచ్చ పచ్చ పచ్చి మట్టి బొమ్మలా


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# plt.scatter(merged['original_line_count'] ,merged['unique_line_count'] )
# plt.show()

In [179]:
# import matplotlib.pyplot as plt
# import numpy as np
# %matplotlib inline

# np.random.seed(42)

# plt.hist(merged['original_word_count'], density=True, bins=30)  # density=False would make counts
# plt.ylabel('Probability')
# plt.xlabel('Data')
# plt.show()

In [177]:
# import matplotlib.pyplot as plt
# plt.plot(merged['original_word_count'],'g*', merged['unique_word_count'], 'ro')
# plt.show()

In [178]:
# import matplotlib.pyplot as plt
# plt.plot(merged['original_line_count'],'g*', merged['unique_line_count'], 'ro')
# plt.show()